In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 64
p1 = .5

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.dropout(x, p=p1)
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 16, 16]          73,856
       BatchNorm2d-4          [-1, 128, 16, 16]             256
   ConvTranspose2d-5             [-1, 64, 8, 8]          73,792
       BatchNorm2d-6             [-1, 64, 8, 8]             128
   ConvTranspose2d-7           [-1, 32, 16, 16]          18,464
       BatchNorm2d-8           [-1, 32, 16, 16]              64
   ConvTranspose2d-9            [-1, 2, 16, 16]             578
Total params: 167,906
Trainable params: 167,906
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.69
Params size (MB): 0.64
Estimated Total Size (MB): 2.34
-------------------------------------------

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00922667 (0.11707193), PSNR 20.34955025 (15.84438529), SSIM 0.44205064 (0.23983822)
Finished training epoch 0


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 72 pixels
  return func(*args, **kwargs)
/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 32 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00774614 (0.00907876), PSNR 21.10914612 (20.74721442), SSIM 0.44679376 (0.50344060)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00407678 (0.00613731), PSNR 23.89682198 (22.22813846), SSIM 0.65437883 (0.59488354)
Finished training epoch 1


/home/konrad/.local/lib/python3.10/site-packages/skimage/_shared/utils.py:394: UserWarning: Color data out of range: Z < 0 in 9 pixels
  return func(*args, **kwargs)


Validate: MSE 0.00442623 (0.00451829), PSNR 23.53965759 (23.60181814), SSIM 0.56659532 (0.63964114)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00289259 (0.00354452), PSNR 25.38713837 (24.53663515), SSIM 0.71954817 (0.67436005)
Finished training epoch 2
Validate: MSE 0.00341758 (0.00307756), PSNR 24.66281128 (25.19173652), SSIM 0.63378322 (0.70137301)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00332642 (0.00291249), PSNR 24.78022766 (25.37720675), SSIM 0.73412955 (0.72114833)
Finished training epoch 3
Validate: MSE 0.00320018 (0.00278640), PSNR 24.94825363 (25.62407907), SSIM 0.67004347 (0.73924966)
Finished validation.
Starting training epoch 4
Epoch: 4, MSE 0.00286830 (0.00276141), PSNR 25.42375565 (25.60990842), SSIM 0.74443018 (0.74992543)
Finished training epoch 4
Validate: MSE 0.00314766 (0.00271727), PSNR 25.02012062 (25.73299528), SSIM 0.69569838 (0.76054004)
Finished validation.
Starting training epoch 5
Epoch: 5, MSE 0.00251450 (0.002717

...

Epoch: 121, MSE 0.00201133 (0.00234019), PSNR 26.96517372 (26.32950878), SSIM 0.76973188 (0.77186555)
Finished training epoch 121
Validate: MSE 0.00279045 (0.00243891), PSNR 25.54325294 (26.20266901), SSIM 0.70668161 (0.77006312)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00277077 (0.00235385), PSNR 25.57399178 (26.30211595), SSIM 0.74945456 (0.77113850)
Finished training epoch 122
Validate: MSE 0.00304001 (0.00285626), PSNR 25.17124748 (25.51916364), SSIM 0.70395207 (0.75826525)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00195387 (0.00234959), PSNR 27.09104347 (26.31416152), SSIM 0.79700279 (0.77148734)
Finished training epoch 123
Validate: MSE 0.00287306 (0.00243832), PSNR 25.41654396 (26.20014257), SSIM 0.71067512 (0.77330575)
Finished validation.
Starting training epoch 124
Epoch: 124, MSE 0.00182143 (0.00234997), PSNR 27.39587212 (26.31329261), SSIM 0.79657346 (0.77151150)
Finished training epoch 124
Validate: MSE 0.00282273 (0.00232

Validate: MSE 0.00285077 (0.00238300), PSNR 25.45037270 (26.29690561), SSIM 0.70433891 (0.77337724)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00240964 (0.00234022), PSNR 26.18047142 (26.33059816), SSIM 0.77112210 (0.77154824)
Finished training epoch 151
Validate: MSE 0.00264920 (0.00236304), PSNR 25.76884842 (26.32750684), SSIM 0.71050012 (0.77001679)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00226831 (0.00234615), PSNR 26.44296837 (26.31684172), SSIM 0.78438210 (0.77112878)
Finished training epoch 152
Validate: MSE 0.00278682 (0.00246815), PSNR 25.54890823 (26.14806141), SSIM 0.71229494 (0.76870495)
Finished validation.
Starting training epoch 153
Epoch: 153, MSE 0.00237026 (0.00234409), PSNR 26.25203323 (26.32228548), SSIM 0.75785935 (0.77130518)
Finished training epoch 153
Validate: MSE 0.00269637 (0.00236133), PSNR 25.69220352 (26.32845584), SSIM 0.70662367 (0.76808253)
Finished validation.
Starting training epoch 154
Epoch: 154, MS

Epoch: 180, MSE 0.00210392 (0.00234409), PSNR 26.76970863 (26.31993032), SSIM 0.77128285 (0.77123535)
Finished training epoch 180
Validate: MSE 0.00269488 (0.00243030), PSNR 25.69461060 (26.20775669), SSIM 0.71007597 (0.77373703)
Finished validation.
Starting training epoch 181
Epoch: 181, MSE 0.00198024 (0.00234508), PSNR 27.03281975 (26.32246117), SSIM 0.77381837 (0.77121258)
Finished training epoch 181
Validate: MSE 0.00298050 (0.00308901), PSNR 25.25711060 (25.19337610), SSIM 0.70622528 (0.75636661)
Finished validation.
Starting training epoch 182
Epoch: 182, MSE 0.00224625 (0.00233888), PSNR 26.48540878 (26.33269779), SSIM 0.77638727 (0.77183981)
Finished training epoch 182
Validate: MSE 0.00283423 (0.00238156), PSNR 25.47564316 (26.29816559), SSIM 0.70923758 (0.77358139)
Finished validation.
Starting training epoch 183
Epoch: 183, MSE 0.00207544 (0.00233039), PSNR 26.82890511 (26.34833054), SSIM 0.77326679 (0.77189875)
Finished training epoch 183
Validate: MSE 0.00279296 (0.00238

Validate: MSE 0.00278504 (0.00231484), PSNR 25.55168152 (26.42173259), SSIM 0.70632714 (0.77252902)
Finished validation.
Starting training epoch 210
Epoch: 210, MSE 0.00227762 (0.00233620), PSNR 26.42519188 (26.33719390), SSIM 0.75273198 (0.77159754)
Finished training epoch 210
Validate: MSE 0.00269324 (0.00235528), PSNR 25.69725037 (26.33307463), SSIM 0.70959997 (0.76655930)
Finished validation.
Starting training epoch 211
Epoch: 211, MSE 0.00210778 (0.00233013), PSNR 26.76175499 (26.34774446), SSIM 0.78888679 (0.77152362)
Finished training epoch 211
Validate: MSE 0.00295217 (0.00254352), PSNR 25.29858398 (26.02523330), SSIM 0.70888025 (0.76939689)
Finished validation.
Starting training epoch 212
Epoch: 212, MSE 0.00202877 (0.00232531), PSNR 26.92768097 (26.35591061), SSIM 0.77573740 (0.77194183)
Finished training epoch 212
Validate: MSE 0.00282635 (0.00236251), PSNR 25.48773766 (26.33393048), SSIM 0.70704114 (0.77025069)
Finished validation.
Starting training epoch 213
Epoch: 213, MS

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00259955 (0.00235463), PSNR 25.85102272 (26.34285553), SSIM 0.71134186 (0.77090939)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()